In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from casadi import *

In [16]:
x = MX.sym('x') # a symbolic variable
objective = (x - 2)**2 # defining the optimization problem
nlp = {'x' : x, 'f' : objective} # creates the optimization problem

solver = nlpsol('solver', 'ipopt', nlp)
solution = solver(x0 = 0) #defining the starting point i gues??
# Print the result
print("Optimal value of x:", solution['x'])
# looking for nlp_f

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  4.0000000e+00 0.00e+00 4.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [18]:
## enzyme kinetics optimisation problem...

In [21]:

# Define constants for the problem
V_max = 1.5   # Maximum reaction rate in mM/s
K_m = 0.5     # Michaelis constant in mM
E_t = 1.0     # Total enzyme concentration in mM

# Define the decision variable: substrate concentration [S]
S = SX.sym('S')

# Define the reaction rate according to Michaelis-Menten equation
v = (V_max * S) / (K_m + S)

# Objective function: We want to maximize the reaction rate v, 
# which is equivalent to minimizing the negative reaction rate -v
objective = -v

# Constraints
# Inequality: 0 <= S <= 10
constraint_ineq = [S >= 0, S <= 10]

# Equality: E_t = E + ES (In this example, we assume that E + ES = E_t = 1 is a simple constraint)
# We don't have an explicit equation for [E] and [ES], so we'll just enforce [S] to be within a feasible region

# Formulate the optimization problem
nlp = {
    'x': S,                 # Decision variable
    'f': objective,         # Objective function
    'g': vertcat(*constraint_ineq)  # Constraints
}

# Choose an NLP solver
solver = nlpsol('solver', 'ipopt', nlp)

# Solve the problem
solution = solver(
    x0=1.0,                # Initial guess for [S]
    lbx=0,                 # Lower bound for [S]
    ubx=10,                # Upper bound for [S]
    lbg=[0, -ca.inf],      # Lower bounds for constraints: S >= 0 (inequality constraint)
    ubg=[ca.inf, 10]       # Upper bounds for constraints: S <= 10 (inequality constraint)
)

# Extract the optimal substrate concentration and the maximum reaction rate
optimal_S = solution['x'].full().flatten()[0]
optimal_v = -solution['f'].full().flatten()[0]

print(f"Optimal substrate concentration [S]: {optimal_S:.2f} mM")
print(f"Maximum reaction rate v: {optimal_v:.2f} mM/s")


This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        1
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        2
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -1.0000000e+00 0.00e+00 3.33e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [25]:
import casadi as ca

# Define constants
F_in = 1.0  # Inflow rate (L/min)
V = 10.0    # Reactor volume (L)
k = 0.1     # Reaction rate constant (1/min)
A_in = 1.0  # Inlet concentration of A (mol/L)

# Define time as a symbolic variable
t = ca.SX.sym('t')

# Define the state variable for concentration of A
A = ca.SX.sym('A')

# Define the differential equation for A
dA_dt = (F_in / V) * (A_in - A) - k * A

# Define the algebraic equation for B
B = A_in - A

# Stack the differential equation into an SX vector
dae = {
    'x': A,         # Differential state (concentration of A)
    'z': B,         # Algebraic state (concentration of B)
    'ode': dA_dt,   # Differential equation
    'alg': ca.vertcat()  # No explicit algebraic equation to solve, but B is defined algebraically
}

# Create an integrator object using cvodes
opts = {'tf': 10.0}  # Integrate from t=0 to t=10 minutes
integrator = ca.integrator('integrator', 'cvodes', dae, opts)

# Initial conditions
A0 = 0.5  # Initial concentration of A (mol/L)
z0 = []   # No initial condition required for algebraic state B

# Integrate the system
sol = integrator(x0=A0, z0=z0)

# Extract the solution
A_sol = sol['xf'].full().flatten()[0]  # Final concentration of A
B_sol = A_in - A_sol  # Final concentration of B

print(f"Final concentration of A: {A_sol:.2f} mol/L")
print(f"Final concentration of B: {B_sol:.2f} mol/L")


CasADi - 2024-08-27 17:55:21 WARNING("The options 't0', 'tf', 'grid' and 'output_t0' have been deprecated.
The same functionality is provided by providing additional input arguments to the 'integrator' function, in particular:
 * Call integrator(..., t0, tf, options) for a single output time, or
 * Call integrator(..., t0, grid, options) for multiple grid points.
The legacy 'output_t0' option can be emulated by including or excluding 't0' in 'grid'.
Backwards compatibility is provided in this release only.") [.../casadi/core/integrator.cpp:515]


RuntimeError: .../casadi/core/function_internal.cpp:146: Error calling CvodesInterface::init for 'integrator':
.../casadi/interfaces/sundials/cvodes_interface.cpp:105: Assertion "nz_==0 && nrz_==0" failed:
CVODES does not support algebraic variables